In [1]:
pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import random
from deap import base
from deap import creator
from deap import tools

In [ ]:
# Types
from deap import base, creator
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [ ]:
np.random.random()

0.9639233108882069

In [ ]:
# Initialization
import random
from deap import tools

IND_SIZE = 10

toolbox = base.Toolbox()
toolbox.register("attribute", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
# Operators
def evaluate(individual):
    return sum(individual),

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

In [ ]:
# Algorithms
def main():
    pop = toolbox.population(n=50)
    CXPB, MUTPB, NGEN = 0.5, 0.2, 40

    # Evaluate the entire population
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(NGEN):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = map(toolbox.clone, offspring)

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring

    return pop

**SOP with DEAP**

In [ ]:
# Import modules
import random

from deap import base
from deap import creator
from deap import tools

ModuleNotFoundError: ignored

In [ ]:
import numpy as np
import pandas as pd
data=pd.read_csv("/content/drive/MyDrive/PhD UCO 062022/step by step/Datasets 12062022/diabetes.csv")
data.head()

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1

In [ ]:
# Creator
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

/usr/local/lib/python3.10/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [ ]:
# Tools
toolbox = base.Toolbox()
# Attribute generator
toolbox.register("attr_bool", random.randint, 0, 1)
# Structure initializers
# set the number of integers the individual has
toolbox.register("individual", tools.initRepeat, creator.Individual,
toolbox.attr_bool, data.shape[0])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from imblearn.metrics import specificity_score
# Evaluation function
def evalOneMax(individual, data):
  dtr=[]
  dts=[]
  for i in range(len(individual)):
    if individual[i]==0:
      dtr.append(data.loc[i].values)
    else:
      dts.append(data.loc[i].values)
  dts=pd.DataFrame(dts, columns=data.columns)
  dtr=pd.DataFrame(dtr,columns=data.columns)
  X_train=dtr.iloc[:,:-1].values
  y_train=dtr.iloc[:,-1].values
  X_test=dts.iloc[:,:-1].values
  y_test=dts.iloc[:,-1].values
  model = XGBClassifier()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  predictions = [round(value) for value in y_pred]
  rec = recall_score(y_test, predictions)
  spec= specificity_score(y_test, predictions)
  return rec, spec

In [ ]:
# Genetic Operators
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
import deap.algorithms

In [ ]:
# Apply NSGA-II
pop = toolbox.population(n=100)
nsga2 = deap.algorithms.eaMuPlusLambda(pop, toolbox, mu=100, lambda_=100, cxpb=0.7, mutpb=0.3, ngen=100)



In [ ]:

# Sort and print the results
sorted_pop = sorted(nsga2[0], key=lambda x: x.fitness.values)
best_instance = sorted_pop[0]
selected_data = data[best_instance == 1]
unselected_data = data[best_instance == 0]
print("Selected instances:")
print(selected_data)
print("Unselected instances:")
print(unselected_data)

In [ ]:
def main(data):
  pop = toolbox.population(n=30)
        # Evaluate the entire population
  fitnesses = list(map(toolbox.evaluate, pop))
  for ind, fit in zip(pop, fitnesses):
      ind.fitness.values = fit
    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
  CXPB, MUTPB = 0.5, 0.2
    # Extracting all the fitnesses of
  fits = [ind.fitness.values[0] for ind in pop]
    # Variable keeping track of the number of generations
  g = 0

    # Begin the evolution
  while max(fits) < 100 and g < 1000:
        # A new generation
      g = g + 1
      print("-- Generation %i --" % g)
        # Select the next generation individuals
      offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
      offspring = list(map(toolbox.clone, offspring))
        # Apply crossover and mutation on the offspring
      for child1, child2 in zip(offspring[::2], offspring[1::2]):
          if random.random() < CXPB:
              toolbox.mate(child1, child2)
              del child1.fitness.values
              del child2.fitness.values

      for mutant in offspring:
          if random.random() < MUTPB:
              toolbox.mutate(mutant)
              del mutant.fitness.values
        # Evaluate the individuals with an invalid fitness
      invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
      fitnesses = map(toolbox.evaluate, invalid_ind)
      for ind, fit in zip(invalid_ind, fitnesses):
          ind.fitness.values = fit
      pop[:] = offspring
        # Gather all the fitnesses in one list and print the stats
      fits = [ind.fitness.values[0] for ind in pop]

      length = len(pop)
      mean = sum(fits) / length
      sum2 = sum(x*x for x in fits)
      std = abs(sum2 / length - mean**2)**0.5

      print("  Min %s" % min(fits))
      print("  Max %s" % max(fits))
      print("  Avg %s" % mean)
      print("  Std %s" % std)

***CODE By Dalia***

In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import deap.algorithms
import deap.base
import deap.creator
import deap.tools

# Set the random seed for reproducibility
random.seed(1)

# Load the data from CSV
data = pd.read_csv("/content/drive/MyDrive/PhD UCO 062022/step by step/Datasets 12062022/diabetes.csv")
print(data.shape)
labels = data["Outcome"]
data = data.drop("Outcome", axis=1)
data = data.values
print("data: ",data.shape)
print("labels: ",labels.shape)
labels.head()


(768, 9)
data:  (768, 8)
labels:  (768,)


0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from imblearn.metrics import specificity_score
import time
# Define the evaluation function
def evaluate(instance):
    instance = np.array(instance)
    selected_data = data[instance == 1]
    selected_labels = labels[instance == 1]
    unselected_data = data[instance == 0]
    unselected_labels = labels[instance == 0]
    if selected_data.shape[0] == 0:
        return 0, 0
    if len(np.unique(selected_labels)) < 2:
        return -1000000, 0

    #cv = StratifiedKFold(n_splits=1, shuffle=True)
    model = XGBClassifier()
    model.fit(selected_data, selected_labels)
    y_pred = model.predict(unselected_data)
    predictions = [round(value) for value in y_pred]
    rec = recall_score(unselected_labels, predictions)
    spec= specificity_score(unselected_labels, predictions)
    #geo_mean = np.sqrt(specificity * rec)
    return rec, spec

# Define the problem
deap.creator.create("FitnessMulti", deap.base.Fitness, weights=(1.0, 1.0))
deap.creator.create("Individual", np.ndarray, fitness=deap.creator.FitnessMulti)

# Initialize the toolbox
toolbox = deap.base.Toolbox()
toolbox.register("attr_int", random.randint, 0, 1)
toolbox.register("individual", deap.tools.initRepeat, deap.creator.Individual, toolbox.attr_int, data.shape[0])
toolbox.register("population", deap.tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("select", deap.tools.selNSGA2)
toolbox.register("mate", deap.tools.cxTwoPoint)
toolbox.register("mutate", deap.tools.mutFlipBit, indpb=0.05)
# Apply NSGA-II
pop = toolbox.population(n=1000)
time0=time.time()
nsga2 = deap.algorithms.eaMuPlusLambda(pop, toolbox, mu=100, lambda_=100, cxpb=0.7, mutpb=0.3, ngen=1000)
deap.algorithms.eaMuPlusLambda()
time1=time.time()
print("time",time1-time0)
# Sort and print the results
sorted_pop = sorted(nsga2[0], key=lambda x: x.fitness.values)
best_instance = sorted_pop[0]
print(best_instance.shape)
selected_data = data[best_instance == 1]
unselected_data = data[best_instance == 0]
print("Selected instances:")
print(selected_data)
print("Unselected instances:")
print(unselected_data)

gen	nevals
0  	1000  
1  	100   
2  	100   
3  	100   
4  	100   
5  	100   
6  	100   
7  	100   
8  	100   
9  	100   
10 	100   
11 	100   
12 	100   
13 	100   
14 	100   
15 	100   
16 	100   
17 	100   
18 	100   
19 	100   
20 	100   
21 	100   
22 	100   
23 	100   
24 	100   
25 	100   
26 	100   
27 	100   
28 	100   
29 	100   
30 	100   
31 	100   
32 	100   
33 	100   
34 	100   
35 	100   
36 	100   
37 	100   
38 	100   
39 	100   
40 	100   
41 	100   
42 	100   
43 	100   
44 	100   
45 	100   
46 	100   
47 	100   
48 	100   
49 	100   
50 	100   
51 	100   
52 	100   
53 	100   
54 	100   
55 	100   
56 	100   
57 	100   
58 	100   
59 	100   
60 	100   
61 	100   
62 	100   
63 	100   
64 	100   
65 	100   
66 	100   
67 	100   
68 	100   
69 	100   
70 	100   
71 	100   
72 	100   
73 	100   
74 	100   
75 	100   
76 	100   
77 	100   
78 	100   
79 	100   
80 	100   
81 	100   
82 	100   
83 	100   
84 	100   
85 	100   
86 	100   
87 	100   
88 	100   
89 	100   

TypeError: ignored

In [ ]:
random.randint(0,1)

In [ ]:
selected_data.shape

(18, 8)

In [ ]:
unselected_data.shape

(148, 8)

In [ ]:
print(best_instance.shape)

NameError: ignored